In [5]:
import networkx as nx
import requests as rq
from routingapp.common.network_stat_utils import get_inter_group_edges, link_info_mn_to_hmap

In [3]:
json_graph = rq.get('http://0.0.0.0:8000/graph').json()
graph = nx.json_graph.node_link_graph(json_graph)
inter_group_edges = get_inter_group_edges(graph)
inter_group_edges

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]

In [8]:
link_with_port = link_info_mn_to_hmap()
s1, s2 = inter_group_edges[0]
p1 = link_with_port[(f's{s1}', f's{s2}')]['port1']
p2 = link_with_port[(f's{s1}', f's{s2}')]['port2']

In [13]:
deltal_port_stat = rq.get('http://0.0.0.0:8080/delta_port_stat').json()
deltal_port_stat
# link_usage_s1 = ...
# link_usage_s2 = ...

[{'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 5,
  'port_no': 2,
  'tx_packets': 21,
  'rx_packets': 17,
  'tx_bytes': 1944,
  'rx_bytes': 1666,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 5,
  'port_no': 3,
  'tx_packets': 9,
  'rx_packets': 8,
  'tx_bytes': 768,
  'rx_bytes': 670,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 5,
  'port_no': 4,
  'tx_packets': 8,
  'rx_packets': 9,
  'tx_bytes': 670,
  'rx_bytes': 768,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 2000000},
 {'dpid': 5,
  'port_no': 5,
  'tx_packets': 11,
  'rx_packets': 10,
  'tx_bytes': 926,
  'rx_bytes': 866,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 2,
  'port_no': 1,
  'tx_packets': 4,
  'rx_packets': 0,
  'tx_bytes': 240,
  'rx_bytes': 0,
  'rx_error': 0,
  'durratio

In [16]:
dps_hmap = {}
for d in deltal_port_stat:
    key = (d['dpid'], d['port_no'])
    dps_hmap[key] = d
    
dps_hmap

{(5, 1): {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 2): {'dpid': 5,
  'port_no': 2,
  'tx_packets': 21,
  'rx_packets': 17,
  'tx_bytes': 1944,
  'rx_bytes': 1666,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 3): {'dpid': 5,
  'port_no': 3,
  'tx_packets': 9,
  'rx_packets': 8,
  'tx_bytes': 768,
  'rx_bytes': 670,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 4): {'dpid': 5,
  'port_no': 4,
  'tx_packets': 8,
  'rx_packets': 9,
  'tx_bytes': 670,
  'rx_bytes': 768,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 2000000},
 (5, 5): {'dpid': 5,
  'port_no': 5,
  'tx_packets': 11,
  'rx_packets': 10,
  'tx_bytes': 926,
  'rx_bytes': 866,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (2, 1): {'dpid': 2,
  'port_no': 1,
  'tx_packets': 4,
  'rx_packets': 0,
  'tx_bytes': 24

In [37]:
def get_controller_list():
  """Fetches controller list from the API and converts string/int keys to int keys.

  Returns:
    A list of dictionaries with integer keys.
  """
  ctrler_list = rq.get('http://0.0.0.0:8000/controller_list').json()

  # Use list comprehension for concise conversion
  new_ctrler_list = [{int(key): value} for ctrler in ctrler_list for key, value in ctrler.items() if key.isdigit()]

  return new_ctrler_list

get_controller_list()


[{0: {'ip': '0.0.0.0', 'port': 6633}}, {1: {'ip': '0.0.0.0', 'port': 6634}}]

In [25]:
test = rq.get('http://0.0.0.0:8000/controller_list').json()
type(test)

list

In [45]:
from routingapp.common.network_stat_utils import get_sw_ctrler_mapping
RYU_PORT = 8080
def get_all_delta_port_stat():
    ''' Get delta port stat of all controllers in the network
    Return:
    hashmap of (dpid, port):{delta_port_stat}
    '''
    deltal_port_stat = []
    ctrler_list = get_controller_list()
    
    for ctrler in ctrler_list:
        for key, value in ctrler.items():
            deltal_port_stat += rq.get(f'http://{value.get("ip")}:{RYU_PORT+key}/delta_port_stat').json()

    for d in deltal_port_stat:
        key = (d['dpid'], d['port_no'])
        dps_hmap[key] = d

    return dps_hmap

get_all_delta_port_stat()

{(5, 1): {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 2): {'dpid': 5,
  'port_no': 2,
  'tx_packets': 27,
  'rx_packets': 26,
  'tx_bytes': 2494,
  'rx_bytes': 2548,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 3): {'dpid': 5,
  'port_no': 3,
  'tx_packets': 9,
  'rx_packets': 11,
  'tx_bytes': 806,
  'rx_bytes': 964,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 0},
 (5, 4): {'dpid': 5,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 11,
  'tx_bytes': 964,
  'rx_bytes': 964,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 5): {'dpid': 5,
  'port_no': 5,
  'tx_packets': 10,
  'rx_packets': 10,
  'tx_bytes': 866,
  'rx_bytes': 866,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (2, 1): {'dpid': 2,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,


In [18]:
get_sw_ctrler_mapping()

{2: 0, 5: 0, 6: 0, 1: 1, 3: 1, 4: 1}